In [ ]:
%reload_ext autoreload
%autoreload 1
%matplotlib inline

Import dependencies

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cobra
import escher

# Load model

Choose from alternatives

In [ ]:
# Yeast 8
model = cobra.io.read_sbml_model("../data/gemfiles/yeast-GEM-BiGG.xml")

In [ ]:
# Enzyme-constrained Yeast 8, batch
# https://github.com/SysBioChalmers/ecModels/tree/main/ecYeastGEM/model
# This is supposed under CI, i.e.
# automatically re-generated and updated when new models are available.
# This model is based on Yeast8.3.4

# Average enzyme saturation factor (sigma) = 0.5
# Total protein content in the cell [g protein/gDw] (Ptot) = 0.5
# Fraction of enzymes in the model [g enzyme/g protein] (f) = 0.5
# https://github.com/SysBioChalmers/GECKO/blob/main/userData/ecYeastGEM/YeastGEMAdapter.m
model = cobra.io.read_sbml_model("../data/gemfiles/ecYeastGEM_batch.xml")

Show model

In [ ]:
model

# Objective function

In the ecYeast8 (batch) model, the objective function -- growth -- is reaction ID `r_2111`.

This reaction is linked to the biomass reaction, ID `r_4041`.

Here, we also see the stoichiometry.  There are five classes of macromolecules: lipids, proteins, carbohydrates, DNA, and RNA.  And there are two other bulk metabolites: cofactor and ion.

In [ ]:
model.reactions.get_by_id('r_2111')

In [ ]:
model.reactions.get_by_id('r_4041')

Medium

In [ ]:
model.medium

In [ ]:
for reaction_id in model.medium.keys():
    print(model.reactions.get_by_id(reaction_id).name)

Remove bounds on glucose uptake and growth rate

In [ ]:
# (no need because bounds are already unrestricted)
# Unrestrict glucose uptake
model.reactions.get_by_id('r_1714').bounds = (-1000.0, 0)
# Unrestrict oxygen uptake (aerobic)
model.reactions.get_by_id('r_1992').bounds = (-1000.0, 0)
# Unrestrict objective function
model.reactions.get_by_id('r_4041').bounds = (0, 1000.0)

Optimise using (vanilla) FBA

In [ ]:
solution = model.optimize()

In [ ]:
model.summary()

In [ ]:
solution['r_0466No1']

Linear reaction coefficients

In [ ]:
cobra.util.solver.linear_reaction_coefficients(model)

# Demand reactions

In [ ]:
model = model_saved

biomass_component_list = [Lipids, Proteins, Carbohydrates, DNA, RNA, Cofactors, Ions]

all_metabolite_ids = [
    biomass_component.metabolite_id
    for biomass_component in biomass_component_list
]

# Create demand reactions for all bulk pseudometabolites
for metabolite_id in all_metabolite_ids:
    model.add_boundary(model.metabolites.get_by_id(metabolite_id), type='demand')
    
# Set exchange reaction bounds
model.reactions.get_by_id('r_1714').bounds = (-1000, 0)
model.reactions.get_by_id('r_2111').bounds = (0, 1000)

In [ ]:
model.reactions.get_by_id('DM_s_1096[c]')

In [ ]:
for biomass_component in biomass_component_list:
    demand_reaction_id = 'DM_' + biomass_component.metabolite_id
    print(f'{demand_reaction_id} ({biomass_component.metabolite_label})')
    model.objective = demand_reaction_id
    pfba_solution = cobra.flux_analysis.pfba(model)
    print(f'Demand rxn flux: {pfba_solution.fluxes[demand_reaction_id]} h-1')
    print(f'Growth rate: {pfba_solution.fluxes["r_4041"]} h-1')
    time = 1/(pfba_solution.fluxes[demand_reaction_id])
    print(f'Time: {time} h')